In [1]:
import numpy as np
import torch 
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset

In [2]:
# Read Diabetes Dataset
_data = pd.read_csv('diabetes.csv')

In [3]:
#preprocessing
# X : extract all rows and columns except last one
# Y : last column for labeling 
# Numpy'a cevirilmesi

X = _data.iloc[:,0:-1]
Y_string = list(_data.iloc[:,-1])
# It is needed to convert string labels to int 
Y_int = []
for s in Y_string:
    if s == 'positive':
        Y_int.append(1)
    else:
        Y_int.append(0)
# It is needed to convert int to numpy array after that we can convert numpy to torch tensors
Y = np.array(Y_int, dtype = 'float64')

In [4]:
# Normalization Part, it is needed to have the features with the same range of values. For exampel (-1,1)
# for that propose x - mean/standar_deviation
# create StandardScaler object from sklearn
_sc = StandardScaler()
X = _sc.fit_transform(X)
# convert numpy array to torch tensor
x = torch.tensor(X)
y = torch.tensor(Y)
# we need add dimention in the y label becasue of chooing right shape, we need a single value in each row
y = y.unsqueeze(1)
print(y.shape)
print(x.shape)

torch.Size([768, 1])
torch.Size([768, 7])


In [5]:
y.shape

torch.Size([768, 1])

In [6]:
#loading data to dataloader for bach processing and shuffling
class Dataset(Dataset):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __getitem__(self,index):
        #getting one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
    
# create object from the Dataset class
_dataset = Dataset(x,y)
print(len(_dataset))

768


In [7]:
train_loader = torch.utils.data.DataLoader(dataset = _dataset, batch_size = 32, shuffle = True)
# the detail info of the Data loader
print("Batch size value is {}".format(len(train_loader)))
for (x,y) in train_loader:
    print("one batch information")
    print("Data: ------->  {}".format(x.shape))
    print("Label: ------->  {}".format(y.shape))
    break

Batch size value is 24
one batch information
Data: ------->  torch.Size([32, 7])
Label: ------->  torch.Size([32, 1])


In [8]:
# building the Model
class MODEL(nn.Module):
    def __init__(self, input_features):
        super(MODEL,self).__init__()
        self._InputLayer = nn.Linear(input_features,5)
        self._2thLayer = nn.Linear(5,4)
        self._3thLayer = nn.Linear(4,3)
        self._OutputLayer = nn.Linear(3,1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self,x):
        out = self._InputLayer(x)
        out = self.tanh(out)
        out = self._2thLayer(out)
        out = self.tanh(out)
        out = self._3thLayer(out)
        out = self.tanh(out)
        out = self._OutputLayer(out)
        out = self.sigmoid(out)
        return out

In [9]:
# creating the network (instance of the MODEL class)
_network = MODEL(x.shape[1])
# in binary cross entropy the input and output should have the same shape
# size_average = True -- the losses are averaged over observation for each minibatch
Loos_func = torch.nn.BCELoss(size_average = True)
_Optimizer = torch.optim.SGD(_network.parameters(), lr= 0.1, momentum= 0.9)
_network

/Users/rayanabri/opt/anaconda3/envs/pytorchE/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


MODEL(
  (_InputLayer): Linear(in_features=7, out_features=5, bias=True)
  (_2thLayer): Linear(in_features=5, out_features=4, bias=True)
  (_3thLayer): Linear(in_features=4, out_features=3, bias=True)
  (_OutputLayer): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (relu): ReLU()
)

In [10]:
_epochs = 500
for epoch in range(500):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Forward Propagation
        _outputs = _network(inputs)
        # loss calculation
        _Loss = Loos_func(_outputs,labels)
        # clear the gradient buffer (you need to do it manually in pytorch)
        _Optimizer.zero_grad()
        # back propagation
        _Loss.backward()
        # update weights W <- W - lr*gradient
        _Optimizer.step()

    # Accuracy Calculation. Since we are using a sigmoid, we will need to perform some thresholding
    OUT = (_outputs>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    _Acc = (OUT == labels).float().mean()
    # print statistics
    print("Epoch {}/{}, Train_Loss: {:.2f}, Train_Accuracy: {:.2f}".format(epoch+1, _epochs, _Loss, _Acc))

Epoch 1/500, Train_Loss: 0.59, Train_Accuracy: 0.72
Epoch 2/500, Train_Loss: 0.56, Train_Accuracy: 0.72
Epoch 3/500, Train_Loss: 0.60, Train_Accuracy: 0.78
Epoch 4/500, Train_Loss: 0.72, Train_Accuracy: 0.66
Epoch 5/500, Train_Loss: 0.52, Train_Accuracy: 0.81
Epoch 6/500, Train_Loss: 0.53, Train_Accuracy: 0.78
Epoch 7/500, Train_Loss: 0.59, Train_Accuracy: 0.62
Epoch 8/500, Train_Loss: 0.49, Train_Accuracy: 0.78
Epoch 9/500, Train_Loss: 0.36, Train_Accuracy: 0.84
Epoch 10/500, Train_Loss: 0.46, Train_Accuracy: 0.75
Epoch 11/500, Train_Loss: 0.44, Train_Accuracy: 0.75
Epoch 12/500, Train_Loss: 0.53, Train_Accuracy: 0.69
Epoch 13/500, Train_Loss: 0.47, Train_Accuracy: 0.84
Epoch 14/500, Train_Loss: 0.51, Train_Accuracy: 0.78
Epoch 15/500, Train_Loss: 0.62, Train_Accuracy: 0.75
Epoch 16/500, Train_Loss: 0.52, Train_Accuracy: 0.66
Epoch 17/500, Train_Loss: 0.55, Train_Accuracy: 0.72
Epoch 18/500, Train_Loss: 0.42, Train_Accuracy: 0.84
Epoch 19/500, Train_Loss: 0.45, Train_Accuracy: 0.78
Ep